# MotionCommander Manual Control

This tutorial notebook let's you use manually steer your drone using Bitcraze's MotionCommander library. MotionCommander is an abstraction library that makes writing Crazyflie movement scripts easy. 

MotionCommander requires some sort of positioning hardware support such as the Flow deck. The library does not have a notion of absolute position (such as GPS) and instead uses velocity setpoints. Due to this, the error in position will accumulate over time.

The API contains a set of primitives that are easy to understand and use, such as “go forward” or “turn around”.

There are two flavors of primitives:
1. Actions that are blocking and return when a motion is completed
2. Actions that starts a motion and return immediately

In the second variation the user has to stop or change the motion when appropriate by issuing new commands.

The MotionCommander can be used as context manager using the with keyword. In this mode of operation takeoff and landing is executed when the context is created/closed.

Reference Documentation: 
* [Step-by-Step: Motion Commander](https://www.bitcraze.io/documentation/repository/crazyflie-lib-python/master/user-guides/sbs_motion_commander/)
* [MotionCommander GitHub](https://github.com/bitcraze/crazyflie-lib-python/blob/master/examples/step-by-step/sbs_motion_commander.py)


## Imports

In [1]:
# Standard imports
import logging
import sys
import time
from threading import Event
from pynput import keyboard
from dotenv import load_dotenv
import os

# Crazyflie imports
import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.positioning.motion_commander import MotionCommander
from cflib.utils import uri_helper

### Global Variables & Configs

In [2]:
load_dotenv()  # take environment variables from .env

# Your radio frequency. This can be found in the Crazyflie client by running "cfclient" in your terminal.
URI = uri_helper.uri_from_env(default=os.getenv("RADIO_URI"))

# --- Global variables
DEFAULT_HEIGHT = 1.0
BOX_LIMIT = 0.5

# Event to signal that the flow deck is attached
deck_attached_event = Event()

# Variable to store the (x, y) position estimates used by the logger
position_estimate = [0, 0]

# Only output errors from the logging framework
logging.basicConfig(level=logging.ERROR)

cf = None
mc = None
flying = False

# Movement speed (m/s)
SPEED = 0.2

In [10]:
def toggle_takeoff_land():
    """Toggle between takeoff and land safely."""
    global flying, mc
    if not flying:
        print("Taking off...")
        try:
            mc.take_off(0.5, 2.0)   # 0.5 m altitude, 2s duration
            flying = True
        except Exception as e:
            print(f"[WARN] Takeoff failed: {e}")
    else:
        print("Landing...")
        try:
            mc.land(0.0, 2.0)       # land smoothly in 2s
            flying = False
        except Exception as e:
            print(f"[WARN] Landing failed: {e}")


def on_press(key):
    global mc, flying
    if hasattr(key, "char") and key.char is not None:
        c = key.char.lower()
        if c == 'w' and flying:
            mc.start_forward(SPEED)
        elif c == 's' and flying:
            mc.start_back(SPEED)
        elif c == 'a' and flying:
            mc.start_left(SPEED)
        elif c == 'd' and flying:
            mc.start_right(SPEED)
    else:
        if key == keyboard.Key.space:
            toggle_takeoff_land()
        elif key == keyboard.Key.esc:
            print("Emergency exit!")
            return False


def on_release(key):
    global mc, flying
    if hasattr(key, "char") and key.char is not None and flying:
        if key.char.lower() in ['w', 'a', 's', 'd']:
            mc.stop()


def main():
    global cf, mc, flying
    # Initialize drivers
    cflib.crtp.init_drivers()

    cf = Crazyflie(rw_cache='./cache')
    with SyncCrazyflie(URI, cf=cf) as scf:
        # Prevent MotionCommander from auto-takeoff
        with MotionCommander(scf, takeoff=False) as commander:
            mc = MotionCommander(scf, default_height=0.3)
            flying = False
            print("Connected to Crazyflie.")
            print("Press SPACE to takeoff/land, WASD to move, ESC to quit.")

            # Start keyboard listener
            with keyboard.Listener(on_press=on_press, on_release=on_release) as listener:
                listener.join()

            # Ensure safe land if script exits while flying
            if flying:
                print("Landing before exit...")
                mc.land()


In [11]:
if __name__ == '__main__':
    from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
    main()

TypeError: MotionCommander.__init__() got an unexpected keyword argument 'takeoff'